# Parser 1.10 - Unsigned Hexadecimal Literals

The goal of this step is to add the ability to recognize [unsigned hexadecimal (base 16)](https://en.wikipedia.org/wiki/Hexadecimal) integers.

Contemporary digital computers most often operate in [binary (base 2)](https://en.wikipedia.org/wiki/Binary_number) at the hardware level. The need to specify values in that base thus arises naturally. In low level languages such as [assembly](https://en.wikipedia.org/wiki/Assembly_language) it is common for numeric values to be specified as strings of '1's and '0's. However beyond a few digits most humans cannot easily distinguish between such values. Additionally, assembly languages tend to be tied to specific hardware architectures and are not portable between computers with different architectures.

More portable high level languages may not support binary numbers as a basic type. Instead numbers expressed as higher powers of two might be allowed. Base 16 essentially groups together four binary digits into a more compact form that humans find much more readable than binary. This also works well with computer memory cells whose fundamental unit size is eight bits, as two hexadecimal digits are sufficient to represent all 256 possible combinations of eight bits.

>[Octal (base 8)](https://en.wikipedia.org/wiki/Octal), which groups bits by threes, is another representation commonly used when the fundamental memory cell size is a [multiple of three](https://en.wikipedia.org/wiki/18-bit_computing).

Unsigned hexadecimal literals require sixteen symbols to stand for sixteen values. Any set of sixteen unique characters will do, but by far the most common convention is the ten decimal characters plus the first six English alphabetic characters in order:

`{ 0,1,2,3,4,5,6,7,8,9,A,B,C,D,E,F }`

The case of the alphabetic characters is generally considered insignificant, so **67A** and **67a** represent the same number.

>We can represent any base up to base 36 by extending this convention to the 26 English language letters (a capability Python [already provides](https://docs.python.org/3.4/library/functions.html?highlight=int#int), which we are ignoring for the same reason we did for unsigned decimal integers). To even higher bases if we are willing to include non-alphanumeric characters or to distinguish between upper- and lower-case characters. These representations are not particularly human readable, but can be useful for [encoding binary data as text](https://en.wikipedia.org/wiki/Binary-to-text_encoding) for transmission over computer networks or storage purposes. [Base 64](https://en.wikipedia.org/wiki/Base64) encoding is one example.

A problem arises because the set of characters we will use includes as a subset all the symbols used for unsigned decimal literals. This means the parser could not tell which base is meant if it encountered a numeric literal expressed using only that subset. For example, the first two-digit number is **10** in both notations, but stands the value 10 in decimal and the value 16 in hexadecimal.

Many programming languages address this problem by decorating numeric literals with additional symbols to distinguish which base is meant. For example, Intel x86 assembly language uses an optional suffix of 'D' for decimal numbers and a required suffix of 'H' for hexadecimal numbers. Thus **10D** represents the value 10, and **10H** the value 16.

>This can be considered an example of *manifest typing*, ie., making it possible to tell what kind of data we’re dealing with simply by looking at it.

For our purposes we’ll adopt the convention pioneered by the [‘C’ programming language](https://en.wikipedia.org/wiki/C_(programming_language)) of prefixing hexadecimal literals with the characters **0X**. Our hexadecimal literals will match the regular expression

`0[xX][0-9a-fA-F]+`

This means *a prefix of '0x' or '0X', followed by one or more hexadecimal characters in either case*. This allows the alphabetic characters of unsigned hexadecimal literals to be expressed in whichever case the user prefers. The parser will have to account for that at some point. This is easy enough and better still, reduces the cognitive load on users by not requiring them to remember an arbitrary and unnecessary case convention.

## Libraries

In [ ]:
import glob            # for directory processing

import re              # for regular expressions

We will use the **glob** library for locating test files used for regression testing. It will not be used in the parser itself.

## User output

In [ ]:
def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual seperator'''
    UIwriteln( '--------------')

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Parser

In [ ]:
versionNumber = '1.10'

# operands accepted:
# - decimal integer literals
# - hexadecimal integer literals

# errors detected:
# - unrecognized input
# - out of range numeric input

# result tuple:
# - (True, number)
# - (False, None)

def PEdoparse(this):
    
    uintMax = 4294967295                  # 2**32-1, for range checking
    
    # convert unsigned literal to internal form
    # - all chars in input known to be legal hexadecimal characters
    # - checks that value is within range
    
    def convertUint(ulit, base):
        
        uint = 0
        
        # isolate the significant portion of 'ulit'
        # - this drops any leading prefix and all leading zeroes
        # - if search fails, then input is all zeroes (and so is value)
        
        p = re.search('[1-9A-F][0-9A-F]*', ulit.upper())
        
        if p != None:
            for digit in p.group():
                digval = '0123456789ABCDEF'.find(digit)
                if uint <= (uintMax - digval)/base:
                    uint =  uint * base + digval
                else:
                    UIerror(f'\'{ulit}\' is out of range')
                    return (False, None)
        
        return (True, uint)
        
    # top level
    
    # unsigned hexadecimal literal ?
    
    if re.fullmatch('0[xX][0-9a-fA-F]+', this):
        return convertUint(this, 16)
        
    # unsigned decimal literal ?
    
    elif re.fullmatch('[0-9]+', this):
        return convertUint(this, 10)
    
    # don't know what it is
    
    else:
        UIerror(f'\'{this}\' not recognized')
        return(False, None)


The function *PEdoparse()* is modified to recognize our unsigned hexadecimal literals. But what should it do with them after that?

One possibility is to write a new function *convertHex()* by duplicating the entire function *convertUint()*, except replacing every reference to base ten with one to base 16. However this is relatively uninteresting because we already know how to do that. It is slightly more challenging to modify *convertUint()* so that the base being converted from is passed as a parameter.

We'll start by modifying *convertUint()* to accept a base parameter. As written the function can convert unsigned numeric literals in any base from two to 16, although here we'll use it only for bases ten and 16.

>The ability to convert from still higher bases can be added by adjusting the arguments of *re.search()* and *find()* to recognize the mapping of additional characters to values.

At the start of conversion, we know that the input literal contains characters allowed in the base we want to convert from. Thus we can safely use a regular expression in *re.search()* that covers the highest base we want to accept. Since the maximum base we will accept is 16, we can use the regular expression

```Python
[1-9A-F][0-9A-F]*
```

Which means *the first non-zero legal hexadecimal symbol together with any succeding legal hexadecimal symbols*. This will skip over the **0X** prefix and any immediately following **0** characters, relieving us from having to deal with them. If it skips over all of them, meaning no match at all, then all of the significant characters in the literal must be **0**. Which means its value is zero, so that's what we'll return.

We'll also take the opportunity to deal with the case problem in *re.search()*. This could have been dealt with by uppercasing hexadecimal literals as part of the call to *convertUint()*. This would have saved a tiny amount of processing power by avoiding uppercasing decimal literals, which never contain alphabetic characters.

>However any base greater than ten requires the precaution of uppercasing. If the abilty to recognize such a base is added, converting it during the call would increase code size because of the duplication. Always doing so at one place within the called function minimizes code size and also relieves a maintenance programmer from having to worry about it.

In the case of unsigned decimal integer literals only the digit characters **0** to **9** will actually be present. It makes no difference to *convertUint*. We can use the expression

```Python
digval = '0123456789ABCDEF'.find(digit)
```

secure in the knowledge that *digval* will never be greater than nine since the characters **A** to **F** are not present in the input literal.

# Evaluator

In [ ]:
# a null evaluator that does nothing

def EEdoeval(this):
    return (True, this)


## Running the parser

In [ ]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

def startUp():
    '''begin execution'''
    global passCnt, failCnt
    UIshow( 'Parser', versionNumber )
    passCnt = failCnt = 0
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser

def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    UIshow( 'Input', this )
    ok, res = PEdoparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = EEdoeval( res )
        if ok:
            UIshow( 'Final Eval', res )
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [ ]:
def parse():
    
    startUp()
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = versionNumber[:len(versionNumber)-3]
    tail = versionNumber[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {versionNumber} vs {this[-12:-4]}')
    
    with open(this) as f:
        data = f.readlines()            # read the whole file
    for line in data:
        test = line.strip()
        if test and test[0] != '#':     # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp()
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp()
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    currFn = makePath('pass', currNum())

    startUp()
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    currFn = makePath('fail',currNum())

    startUp()
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)               
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

### Testing the parser

Now that there is more than one functioning parser, it is useful to add regression to testing.

We have added one new function to the batch processing section:

- *regress()* takes no arguments. It tests the parser against the current and all previous pass and fail test files. The purpose is to make sure that no change we have made breaks anything that used to work. The name of any test with an unexpected result is listed after the end of all tests.

We have modified two functions to make the parser capable of being run against specific tests:

- *good()* takes one optional argument. If no argument is supplied, then by default *good()* runs the parser against the pass test designed for it. If an argument is supplied, it should be the four digit number of the pass test to run the parser against. This can be any test for any reason, but is meant for the case when *regress()* reported the unfortunate result that a pass test got broken. This provides a simple means of checking to see if a bug fix works.


- *bad()* takes one optional argument. It has the same characteristics and purpose as for *good()*, except for fail tests.

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous and current tests